<a href="https://colab.research.google.com/github/fuman/nlp/blob/main/createKM_Mix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1단계: 환경 설정

In [ ]:
!pip install keybert
!pip install nltk
!pip install sklearn
!pip install konlpy  # 한국어 처리를 위한 라이브러리
!pip install kiwipiepy

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


# 2단계: 라이브러리 IMPORT

In [ ]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt  # 한국어 형태소 분석기
nltk.download('averaged_perceptron_tagger')

# 기존 임포트문들도 유지
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import numpy as np

from kiwipiepy import Kiwi

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# 3단계: 불용어 처리

## 3-1: KIWI 불용어 처리

In [ ]:
def get_keywords_kiwi(text, top_n=5):

    kiwi = Kiwi()

    filtered_tokens = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        #if len(token) != 1 and pos.startswith('N') or pos.startswith('VV') or pos.startswith('SL'):
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            filtered_tokens.append(token)

    filtered_text = ' '.join(filtered_tokens)

    print(filtered_text)

    kw_model = KeyBERT()

    keywords = kw_model.extract_keywords(filtered_text, keyphrase_ngram_range=(1, 2), top_n=top_n, stop_words='english')
    #keywords = kw_model.extract_keywords(filtered_text, top_n=top_n, stop_words='english')
    print(keywords)
    return [keyword for keyword, _ in keywords]

## 3-2: Okt 불용어 처리

In [ ]:
def get_keywords(text, top_n=5, lang='ko'):

    #vectorizer = CountVectorizer(ngram_range=(1, 2))  # 2-gram까지 고려
    #kw_model = KeyBERT(vectorizer=vectorizer)

    kw_model = KeyBERT()

    if lang == 'en':
        # 영어 텍스트 처리
        tokens = word_tokenize(text.lower())
        pos_tags = pos_tag(tokens)
        filtered_tokens = [word for word, pos in pos_tags if pos.startswith('NN') or pos.startswith('VB')]
        filtered_text = ' '.join(filtered_tokens)
    else:
        # 한국어 텍스트 처리
        okt = Okt()
        pos_tags = okt.pos(text)
        #filtered_tokens = [word for word, pos in pos_tags if pos in ['Noun', 'Verb']]
        filtered_tokens = [word for word, pos in pos_tags if pos in ['Noun']]
        filtered_text = ' '.join(filtered_tokens)

    print(filtered_text)

    keywords = kw_model.extract_keywords(filtered_text, keyphrase_ngram_range=(1, 2), top_n=top_n, stop_words='english')
    #keywords = kw_model.extract_keywords(filtered_text, top_n=top_n, stop_words='english')
    print(keywords)
    return [keyword for keyword, _ in keywords]

# 4단계: 핵심키워드 추출

# 5단계: 문서 Load

In [ ]:
import pandas as pd

# 여러 파일 읽기
file_list = ['문서1.txt', '문서2.txt', '문서3.txt', '문서4.txt']  # 파일 이름 목록
df_okt = pd.DataFrame(columns=['파일명', '키워드_한글', '키워드_영어'])
for file_name in file_list:
    with open("/content/drive/MyDrive/01.교육/2024 자연어처리/DAY01/txt/" + file_name, 'r', encoding='utf-8') as file:
        content = file.read()

        keywords_ko = get_keywords(content, 5, 'ko')
        keywords_en = get_keywords(content, 5, 'en')

        #df = df.append({'파일명': file_name, '내용': content}, ignore_index=True)
        new_row = pd.DataFrame({'파일명': [file_name], '키워드_한글': [keywords_ko], '키워드_영어': [keywords_en]})
        df_okt = pd.concat([df_okt, new_row], ignore_index=True)

서울 시청역 인근 명 사망 대형 교통사고 고령 운전자 역주행 인도 돌진 보행자 친 보도 등 종합 시청역 인근 일방 통행 차선 도로 세종 길 를 역주행 차량 도로 다른 대의 차량 연달 후 횡단보도 인도 쪽 돌진 신호 보행자 명 명 사고 남성 명 남성 명 남성 명 남성 명과 남성 명 여성 명 남성 명 부상 가해 운전자 세 남성 현장 검거 음주운전 혐의 것 해당 고령자 급발진 주장 것 다만 급발진 주장 것 대해 목격자 급발진 절대 면서 급발진 때 때 횡단보도 앞 차량 고 연합뉴스 인근 차량 블랙박스 영상 급발진 이 의견 표 이로 가해자 특성 고령 운전자 대한 비판 트위터 등 실시간 현장 사진 전하 누리꾼 노인 운전자 우리 생명 위협 제발 운전면허 테 주지 세 한번 세 개월 한번 세 개월 한번 선 해 운전 미숙 기타 문제 발생 면허 의견 개진
[('블랙박스 영상', 0.469), ('운전자 역주행', 0.4491), ('역주행 인도', 0.4299), ('남성 명과', 0.4238), ('서울 시청역', 0.4194)]
시청역 인근에서 사망이란 대형 교통사고가 났다 고령 역주행을 하다 인도로 돌진해 보행자들을 덮친 사고였다 보도 종합하면 시청역 인근 일방통행인 도로 를 역주행한 차량이 도로의 다른 차량을 연달아 들이받은 후 횡단보도가 있는 인도 쪽으로 돌진해 신호를 기다리던 보행자들을 덮쳐 다쳤다 남성 남성 숨졌다 남성 여성 남성 부상을 당했다 가해 현장 검거됐다 음주운전 없는 것으로 전해졌고 해당 고령자는 급발진을 주장하고 있는 것으로 알려졌다 다만 주장하고 있는 것에 대해 목격자들은 “ 급발진은 절대 아니었다 ” 면서 “ 급발진을 할 때는 끝날 때까지 박았어야 했는데 횡단보도 앞에서 차량이 멈췄다 뭐라도 했다 ” 고 연합뉴스에 밝혔다 인근 블랙박스 영상이 퍼지자 급발진이 아니었다고 보는 이들의 의견도 속속 표출됐다 이로 가해자 특성인 ‘ 고령 ’ 운전자에 대한 비판이 이어졌다 x 트위터 등에선 현장 사진을 전하는 누리꾼들이 “ 노인 운전자들이 우리 생명에 위협을 

In [ ]:
import pandas as pd

# 여러 파일 읽기
file_list = ['문서1.txt', '문서2.txt', '문서3.txt', '문서4.txt']  # 파일 이름 목록
df_wiki = pd.DataFrame(columns=['파일명', '키워드'])
for file_name in file_list:
    with open("/content/drive/MyDrive/01.교육/2024 자연어처리/DAY01/txt/" + file_name, 'r', encoding='utf-8') as file:
        content = file.read()

        keywords_ko = get_keywords_kiwi(content, 5)

        #df = df.append({'파일명': file_name, '내용': content}, ignore_index=True)
        new_row = pd.DataFrame({'파일명': [file_name], '키워드': [keywords_ko]})
        df_wiki = pd.concat([df_wiki, new_row], ignore_index=True)

서울 시청역 인근 사망 교통 사고 고령 운전자 역주행 인도 돌진 보행자 사고 보도 종합 시청역 인근 일방통행 차선 도로 세종대로 역주행 차량 도로 차량 횡단보도 인도 돌진 신호 보행자 사고 남성 남성 남성 남성 남성 여성 남성 부상 가해 운전자 남성 현장 검거 음주 운전 혐의 해당 고령자 급발진 주장 급발진 주장 목격자 급발진 급발진 횡단보도 차량 연합뉴스 인근 차량 블랙박스 영상 급발진 의견 표출 가해자 특성 고령 운전자 비판 X 트위터 실시간 현장 사진 누리 노인 운전자 우리 생명 위협 운전면허 아무 개월 개월 선별 운전 기타 문제 발생 면허 의견 개진
[('인근 일방통행', 0.4873), ('블랙박스 영상', 0.4588), ('일방통행 차선', 0.4579), ('세종대로 역주행', 0.4417), ('운전자 역주행', 0.4225)]
김포시 시장 김병수 본격 AI 시대 대비 관내 지능 교통 장비 확대 지능 교통 체계 ITS 확대 사업 국비 포함 투입 관내 주요 도로 첨단 AI 지능 교통 장비 설치 급속 도시 발전 교통량 증가 교통 혼잡 완화 교통 안전 강화 추진 사업 시작 투입 관내 개소 첨단 신호 제어 교통 정보 수집 제공 첨단 지능 교통 장비 설치 확대 구축 사업 투입 관내 개소 AI 지능 교통 정보 인프라 확대 구축 이후 교통 신호등 온라인 구축 사업 추진 긴급 차량 신호 사업 추진 계획 향후 자율 주행 시대 대비 차세대 지능 교통 체계 C ITS 구축 계획 시민 교통 편의 다각 정책 구상 진석 도시 안전 정보 센터장 김포 대비 다양 교통 정보 정책 계획 추진 시민 편리 교통 환경 노력
[('교통 환경', 0.4168), ('완화 교통', 0.4062), ('교통 신호등', 0.4006), ('김병수 본격', 0.3822), ('안전 강화', 0.3794)]
기술 단어 문장 상황 사용 가지 해석 가능 단어 기술 과학 기술 발전 한자 의미 사용 한자어 상황 Skill Technique Technology 다양 형태 번역 단어 오늘 우리 기술 한자

# 6단계: 실행

In [ ]:
df_okt

,파일명,키워드_한글,키워드_영어
0,문서1.txt,"[블랙박스 영상, 운전자 역주행, 역주행 인도, 남성 명과, 서울 시청역]","[블랙박스 영상이, 시청역 인근에서, 종합하면 시청역, 인근 일방통행인, 이들의 의견도]"
1,문서2.txt,"[교통혼잡 완화, 이후 교통신호등, 교통 환경, 완화 교통, 교통신호등]","[완화 교통안전을, 편리한 교통환경을, 지능형교통장비를 설치해, 교통안전을 강화하기..."
2,문서3.txt,"[국어사전 정의, 문장 상황, 생산 과정, 생활 상의, 건축 용어사전]","[육체노동이나 정신노동의, 국어사전의 정의는, 인간의 생산적, 용구나 생활상의, 조..."
3,문서4.txt,[],"[drivers koreans, korea drivers, seoul drove, ..."


In [ ]:
df_wiki

,파일명,키워드
0,문서1.txt,"[인근 일방통행, 블랙박스 영상, 일방통행 차선, 세종대로 역주행, 운전자 역주행]"
1,문서2.txt,"[교통 환경, 완화 교통, 교통 신호등, 김병수 본격, 안전 강화]"
2,문서3.txt,"[국어사전 정의, 임석진 윤용, 번역 skill, 상황 skill, 용구 생활상]"
3,문서4.txt,"[accidents younger, korea drivers, seoul drove..."
